# **Datasets & DataLoaders**

Code for processing data samples can get messy and hard to maintain; we ideally want our dataset code to be decoupled from our model training code for better readability and modularity.

> PyTorch provides two data primitives:
> * `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`
>
> that allow you to use pre-loaded datasets as well as your own data.
>
> `Dataset` stores the samples and their corresponding labels, and `DataLoader` wraps an iterable around the Dataset to enable easy access to the samples.